In [1]:
import pandas as pd

df = pd.read_csv('../input/historical_transactions.csv', usecols=['card_id','purchase_date','month_lag','merchant_id'], parse_dates=['purchase_date'])
df['year'] = df['purchase_date'].dt.year
df['month'] = df['purchase_date'].dt.month
df['ym'] = (df['year'] - 2017) * 12 + df['month']
df['reference_month'] = df['ym'] - df['month_lag']
rf = df[['card_id','reference_month']].groupby('card_id').max().reset_index()
print('\nReference months:')
print(rf.head())

dfx = pd.read_csv('../input/train.csv')
dfx['out'] = dfx['target'].apply(lambda x: 1 if x < -20 else 0)

dfx = dfx.merge(rf, how='left')
dfx['int_first_active_month_reference_month'] = dfx['first_active_month'].astype(str) + '_' + dfx['reference_month'].astype(str)

dfx['fam'] = dfx['first_active_month'].apply(lambda x: (10 + int(x.split('-')[0]) - 2017) * 12 + int(x.split('-')[1]))
dfx['far'] = dfx['reference_month'].apply(lambda x: x + 120)
dfx['fad'] = dfx['far'] - dfx['fam']

dfx['first_active_month'] = dfx['int_first_active_month_reference_month']

print('\nStatistics:')
print(dfx.head())

df = dfx.groupby('first_active_month')['out'].sum().reset_index()
df['reference_month'] = df['first_active_month'].apply(lambda x: x.split('_')[1]).astype(int)
df['month'] = df['first_active_month'].apply(lambda x: x.split('_')[0])

dff = df.groupby('month')['reference_month'].min().reset_index()
dff.columns = ['month','refmin']
df = df.merge(dff, how='left')

dff = df.groupby('month')['reference_month'].max().reset_index()
dff.columns = ['month','refmax']
df = df.merge(dff, how='left')

df['refmin'] = df['reference_month'] - df['refmin']
df['refmax'] = df['refmax'] - df['reference_month']

dfx = dfx.merge(df[['first_active_month','refmin','refmax','month']], how='left')
dfx['type'] = 0

print('\nGrouped statistics:')
dfx.drop(['feature_1','feature_2','feature_3','int_first_active_month_reference_month'], axis=1, inplace=True)
print(dfx.head())

selguarantee = (dfx['month'].isin(['2018-02','2018-01','2017-12','2017-11'])) | (dfx['refmin'].isin([0,1])) #| (dfx['fad']<3)
sel = (dfx['month'].isin(['2018-02','2018-01','2017-12','2017-11','2017-10'])) | (dfx['refmin'].isin([0,1,2])) #| (dfx['fad']<4)
sel2 = (dfx['refmax'].isin([0]))

dfx.loc[sel2, 'type'] = 2 #no outliers mostly
dfx.loc[sel, 'type'] = 1 #no outliers mostly
dfx.loc[selguarantee, 'type'] = 10 #no outliers with guarantee

print('\nOutliers mean/sum in group and total size of group:')
print(dfx.groupby('type')['out'].agg(['mean','sum','size']).reset_index())


Reference months:
           card_id  reference_month
0  C_ID_00007093c1               14
1  C_ID_0001238066               14
2  C_ID_0001506ef0               14
3  C_ID_0001793786               10
4  C_ID_000183fdda               14

Statistics:
  first_active_month          card_id  feature_1  feature_2  feature_3  \
0         2017-06_14  C_ID_92a2005557          5          2          1   
1         2017-01_13  C_ID_3d0044924f          4          1          0   
2         2016-08_14  C_ID_d639edf6cd          2          2          0   
3         2017-09_14  C_ID_186d6a6901          4          3          0   
4         2017-11_14  C_ID_cdbd2c0db2          1          3          0   

     target  out  reference_month int_first_active_month_reference_month  fam  \
0 -0.820283    0               14                             2017-06_14  126   
1  0.392913    0               13                             2017-01_13  121   
2  0.688056    0               14                             20